In [1]:
! pip3 install s3fs

  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached botocore-1.34.131-py3-none-any.whl.metadata (5.7 kB)
Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
Using cached botocore-1.34.131-py3-none-any.whl (12.3 MB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec None
error: uninstall-no-record-file

× Cannot uninstall fsspec None
╰─> The package's contents are unknown: no RECORD file was found for fsspec.

hint: You might be able to recover from this via: pip install --force-reinstall --no-deps fsspec==2023.6.0


# import libraries

In [2]:
import boto3
import pandas as pd 

from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


# import input data from S3

In [3]:

role = get_execution_role()

bucket = 'mysagemakerprojects'

data_key = 'cancer-prediction/test_data.csv'

data_location = f's3://{bucket}/{data_key}'

cancer_data = pd.read_csv(data_location)

cancer_data.head(10)

,age,gender,cancer_type,cancer_stage,outcome
0,27,male,colon,4,survived
1,78,female,breast,1,survived
2,42,female,breast,3,survived
3,38,female,breast,4,died
4,46,female,breast,2,survived
5,73,female,breast,2,survived
6,11,male,lung,3,survived
7,60,male,colon,2,died
8,100,female,colon,1,survived
9,42,male,lung,4,died


In [4]:
cancer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   age           1000 non-null   int64 
 1   gender        1000 non-null   object
 2   cancer_type   1000 non-null   object
 3   cancer_stage  1000 non-null   int64 
 4   outcome       1000 non-null   object
dtypes: int64(2), object(3)
memory usage: 39.2+ KB


In [5]:
cancer_data.isna().sum()

age             0
gender          0
cancer_type     0
cancer_stage    0
outcome         0
dtype: int64

In [6]:
print(cancer_data['outcome'].unique())

['survived' 'died']


In [7]:
cancer_data['outcome'] = cancer_data['outcome'].map({'died': 0, 'survived': 1})


In [8]:
cancer_data.head()

,age,gender,cancer_type,cancer_stage,outcome
0,27,male,colon,4,1
1,78,female,breast,1,1
2,42,female,breast,3,1
3,38,female,breast,4,0
4,46,female,breast,2,1


In [9]:
cancer_data.columns

Index(['age', 'gender', 'cancer_type', 'cancer_stage', 'outcome'], dtype='object')

In [10]:
# Create the OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
encoder = OneHotEncoder()

for i in ['gender', 'cancer_type']:
    # Fit and transform the data
    encoded_data = encoder.fit_transform(cancer_data[[i]])
    
    # Create a new DataFrame from the encoded data
    encoded_df = pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out([i]))
    
    # Join the encoded DataFrame with the original DataFrame
    cancer_data = cancer_data.join(encoded_df)

In [11]:
cancer_data.head()

,age,gender,cancer_type,cancer_stage,outcome,gender_female,gender_male,cancer_type_breast,cancer_type_colon,cancer_type_lung,cancer_type_prostate
0,27,male,colon,4,1,0.0,1.0,0.0,1.0,0.0,0.0
1,78,female,breast,1,1,1.0,0.0,1.0,0.0,0.0,0.0
2,42,female,breast,3,1,1.0,0.0,1.0,0.0,0.0,0.0
3,38,female,breast,4,0,1.0,0.0,1.0,0.0,0.0,0.0
4,46,female,breast,2,1,1.0,0.0,1.0,0.0,0.0,0.0


In [12]:
cancer_data.head()

,age,gender,cancer_type,cancer_stage,outcome,gender_female,gender_male,cancer_type_breast,cancer_type_colon,cancer_type_lung,cancer_type_prostate
0,27,male,colon,4,1,0.0,1.0,0.0,1.0,0.0,0.0
1,78,female,breast,1,1,1.0,0.0,1.0,0.0,0.0,0.0
2,42,female,breast,3,1,1.0,0.0,1.0,0.0,0.0,0.0
3,38,female,breast,4,0,1.0,0.0,1.0,0.0,0.0,0.0
4,46,female,breast,2,1,1.0,0.0,1.0,0.0,0.0,0.0


In [13]:
df = cancer_data.drop(['cancer_type', 'gender'], axis=1)


In [14]:
df.head()

,age,cancer_stage,outcome,gender_female,gender_male,cancer_type_breast,cancer_type_colon,cancer_type_lung,cancer_type_prostate
0,27,4,1,0.0,1.0,0.0,1.0,0.0,0.0
1,78,1,1,1.0,0.0,1.0,0.0,0.0,0.0
2,42,3,1,1.0,0.0,1.0,0.0,0.0,0.0
3,38,4,0,1.0,0.0,1.0,0.0,0.0,0.0
4,46,2,1,1.0,0.0,1.0,0.0,0.0,0.0


In [15]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import pandas as pd

# Define S3 bucket
bucket = 'mysagemakerprojects'
prefix = 'cancer-prediction'
role = 'sagemakeruser'

# SageMaker session
sagemaker_session = sagemaker.Session()


In [16]:
# Load dataset (assuming you have uploaded the dataset to S3)
# s3 = boto3.client('s3')
# s3.download_file(bucket, f'{prefix}/cancer_data.csv', 'cancer_data.csv')

# Read the dataset
data = df.copy()

# Split data into train and test
from sklearn.model_selection import train_test_split

X = data.drop('outcome', axis=1)  # Features
y = data['outcome']  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Upload train/test data back to S3
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

# Upload the data to S3 for training
train_s3_uri = sagemaker_session.upload_data('train_data.csv', bucket=bucket, key_prefix=f'{prefix}/train')
test_s3_uri = sagemaker_session.upload_data('test_data.csv', bucket=bucket, key_prefix=f'{prefix}/test')


In [17]:
! ls -l

total 80
-rw-r--r-- 1 sagemaker-user users 17244 Oct  3 04:54 cancer_prediction.ipynb
-rw-r--r-- 1 sagemaker-user users  3493 Oct  2 17:43 test.csv
-rw-r--r-- 1 sagemaker-user users  6319 Oct  3 04:54 test_data.csv
-rw-r--r-- 1 sagemaker-user users 14108 Oct  2 17:43 train.csv
-rw-r--r-- 1 sagemaker-user users  1406 Oct  3 04:52 train.py
-rw-r--r-- 1 sagemaker-user users 24903 Oct  3 04:54 train_data.csv


In [18]:
%%writefile train.py
import argparse
import os
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import joblib

def model_fn(model_dir):
    """Load the model for inference"""
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    return model

if __name__ == "__main__":
    parser = argparse.ArgumentParser()

    # SageMaker specific arguments
    parser.add_argument('--output-data-dir', type=str, default=os.getenv('SM_OUTPUT_DATA_DIR'))
    parser.add_argument('--model-dir', type=str, default=os.getenv('SM_MODEL_DIR'))
    parser.add_argument('--train-dir', type=str, default=os.getenv('SM_CHANNEL_TRAIN'))

    args = parser.parse_args()

    # Load the training data
    train_data = pd.read_csv(os.path.join(args.train_dir, 'train_data.csv'))
    
    # Separate the target from the features
    X = train_data.drop('outcome', axis=1)
    y = train_data['outcome']
    
    # Train the Linear Regression model
    model = LinearRegression()
    model.fit(X, y)

    # Save the model to the model directory so SageMaker can upload it to S3
    joblib.dump(model, os.path.join(args.model_dir, "model.joblib"))
    
    # Test the model on the training data
    y_pred = model.predict(X)
    mse = mean_squared_error(y, y_pred)
    print(f'Mean Squared Error on Training Data: {mse}')


Overwriting train.py


In [19]:
prefix

'cancer-prediction'

In [20]:
script_path = 'train.py'

In [21]:
train_data.head()

,age,cancer_stage,gender_female,gender_male,cancer_type_breast,cancer_type_colon,cancer_type_lung,cancer_type_prostate,outcome
214,47,2,1.0,0.0,0.0,1.0,0.0,0.0,0
357,19,1,0.0,1.0,0.0,0.0,1.0,0.0,1
525,71,1,1.0,0.0,1.0,0.0,0.0,0.0,1
180,61,2,0.0,1.0,0.0,0.0,1.0,0.0,0
975,27,1,0.0,1.0,0.0,0.0,0.0,1.0,1


In [22]:
train_s3_uri

's3://mysagemakerprojects/cancer-prediction/train/train_data.csv'

In [23]:
from sagemaker.sklearn.estimator import SKLearn

# Define the SKLearn estimator
sklearn = SKLearn(
    entry_point=script_path,               # path to the training script ('train.py')
    role='sagemakeruser',                  # IAM role with permissions
    instance_count=1,                      # number of instances
    instance_type='ml.m5.large',           # type of instance
    framework_version='0.23-1',            # scikit-learn version
    py_version='py3',                      # Python version
    output_path=f's3://{bucket}/{prefix}/output',  # output path for model artifacts
    sagemaker_session=sagemaker.Session()  # session to manage interactions with SageMaker
)

# Set up the training input data
train_input = train_s3_uri # TrainingInput(s3_data=f's3://{bucket}/{prefix}/train_data.csv', content_type='csv')

# Train the model
sklearn.fit({'train': train_input})


INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2024-10-03-04-54-07-902


2024-10-03 04:54:08 Starting - Starting the training job...
2024-10-03 04:54:29 Starting - Preparing the instances for training...
2024-10-03 04:55:00 Downloading - Downloading input data...
2024-10-03 04:55:25 Downloading - Downloading the training image...
2024-10-03 04:56:11 Training - Training image download completed. Training in progress..2024-10-03 04:56:15,317 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-10-03 04:56:15,320 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-10-03 04:56:15,375 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-10-03 04:56:15,526 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-10-03 04:56:15,539 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-10-03 04:56:15,552 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-10-03 04:

In [24]:
test_data.head()

,age,cancer_stage,gender_female,gender_male,cancer_type_breast,cancer_type_colon,cancer_type_lung,cancer_type_prostate,outcome
788,82,1,1.0,0.0,1.0,0.0,0.0,0.0,1
199,50,4,1.0,0.0,0.0,1.0,0.0,0.0,0
130,94,1,1.0,0.0,1.0,0.0,0.0,0.0,1
124,29,2,1.0,0.0,1.0,0.0,0.0,0.0,1
717,86,3,1.0,0.0,1.0,0.0,0.0,0.0,1


In [25]:
from sagemaker.model import ModelPackage

# Register the trained model
model_package_group_name = 'YourModelPackageGroupName'

# Register the model in the SageMaker Model Registry
model_package = sklearn.register(
    content_types=["text/csv"],                       # input content type
    response_types=["text/csv"],                      # output content type
    inference_instances=["ml.m5.large"],              # instance for inference
    transform_instances=["ml.m5.large"],              # instance for batch transform jobs
    model_package_group_name=model_package_group_name # model package group name for versioning
)

print(f"Model package ARN: {model_package.model_package_arn}")


Model package ARN: arn:aws:sagemaker:us-east-1:390403890405:model-package/YourModelPackageGroupName/2


In [26]:
# Deploy the trained model to an endpoint
sklearn_predictor = sklearn.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large'  # instance type for endpoint
)

# Making predictions using the deployed model
predictions = sklearn_predictor.predict(test_data)

# Clean up the endpoint after use
sklearn_predictor.delete_endpoint()


INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2024-10-03-04-56-55-929
INFO:sagemaker:Creating endpoint-config with name sagemaker-scikit-learn-2024-10-03-04-56-55-929
INFO:sagemaker:Creating endpoint with name sagemaker-scikit-learn-2024-10-03-04-56-55-929


------!

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from primary with message "<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/sagemaker-scikit-learn-2024-10-03-04-56-55-929 in account 390403890405 for more information.

In [29]:
import boto3

# Initialize a SageMaker client
sagemaker_client = boto3.client('sagemaker')

# List all the endpoints in your account
response = sagemaker_client.list_endpoints()

# Print the list of endpoints
for endpoint in response['Endpoints']:
    print(f"Endpoint Name: {endpoint['EndpointName']}, Status: {endpoint['EndpointStatus']}")


Endpoint Name: Endpoint-20241003-050358, Status: Creating
Endpoint Name: sagemaker-scikit-learn-2024-10-03-04-56-55-929, Status: InService
Endpoint Name: sagemaker-scikit-learn-2024-10-03-04-47-13-780, Status: InService
Endpoint Name: sagemaker-scikit-learn-2024-10-03-04-42-50-747, Status: InService
Endpoint Name: sagemaker-scikit-learn-2024-10-03-04-38-06-267, Status: InService
Endpoint Name: sagemaker-scikit-learn-2024-10-03-04-31-04-972, Status: InService


In [27]:
import boto3

# Initialize SageMaker client
sm_client = boto3.client('sagemaker')

# Create a model package group
model_package_group_name = "CancerPredictionModelPackageGroup"

sm_client.create_model_package_group(
    ModelPackageGroupName=model_package_group_name,
    ModelPackageGroupDescription="Model group for cancer prediction model"
)


{'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:390403890405:model-package-group/CancerPredictionModelPackageGroup',
 'ResponseMetadata': {'RequestId': '36ae1d9a-3a8c-4787-aadd-ccc93880c640',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '36ae1d9a-3a8c-4787-aadd-ccc93880c640',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '121',
   'date': 'Thu, 03 Oct 2024 05:07:48 GMT'},
  'RetryAttempts': 0}}

In [31]:
# Get the model artifact path from the trained model
model_artifact_s3 = sklearn.model_data  # model_data returns the S3 location of the trained model

# Register the model in the Model Registry
model_package = sm_client.create_model_package(
    ModelPackageGroupName=model_package_group_name,
    ModelPackageDescription="Cancer prediction model using linear regression",
    ModelApprovalStatus="PendingManualApproval",  # Set initial status to pending for review
    InferenceSpecification={
        "Containers": [
            {
                "Image": "683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3",  # scikit-learn image
                "ModelDataUrl": model_artifact_s3
            }
        ],
        "SupportedContentTypes": ["text/csv"],
        "SupportedResponseMIMETypes": ["text/csv"]
    }
)

print(f"Model Package ARN: {model_package['ModelPackageArn']}")


Model Package ARN: arn:aws:sagemaker:us-east-1:390403890405:model-package/CancerPredictionModelPackageGroup/1


In [32]:
response = sm_client.list_model_packages(
    ModelPackageGroupName=model_package_group_name
)

for model_package_summary in response['ModelPackageSummaryList']:
    print(f"Version: {model_package_summary['ModelPackageVersion']}, Status: {model_package_summary['ModelApprovalStatus']}")


Version: 1, Status: PendingManualApproval


In [33]:
model_package_arn = model_package['ModelPackageArn']  # ARN from previous registration

sm_client.update_model_package(
    ModelPackageArn=model_package_arn,
    ModelApprovalStatus="Approved"  # This can be set to "Approved" for staging/production
)


{'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:390403890405:model-package/CancerPredictionModelPackageGroup/1',
 'ResponseMetadata': {'RequestId': '58c8cd81-5f7f-47b3-9a9f-c25f0490e49b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '58c8cd81-5f7f-47b3-9a9f-c25f0490e49b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '112',
   'date': 'Thu, 03 Oct 2024 05:10:53 GMT'},
  'RetryAttempts': 0}}

In [34]:
from sagemaker import ModelPackage

# Load the model package
model_package_arn = model_package['ModelPackageArn']

model = ModelPackage(
    role=role,  # IAM role
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker.Session()
)

# Deploy the model to an endpoint
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    endpoint_name="CancerPredictionEndpoint"
)


INFO:sagemaker:Creating model with name: CancerPredictionModelPackageGroup-2024-10-03-05-11-10-508
INFO:sagemaker:Creating endpoint-config with name CancerPredictionEndpoint
INFO:sagemaker:Creating endpoint with name CancerPredictionEndpoint


-----------------------

KeyboardInterrupt: 